# Personalized UI Recommendation System - Colab Version

## 1. 패키지 설치 및 프로젝트 클론

In [ ]:
# 필수 패키지 설치
!pip install lightgbm pandas numpy scikit-learn joblib pyyaml tqdm

# 코랩 파일 다운로드 활성화
from google.colab import files
print("패키지 설치 완료")

import sys
import os

# 프로젝트 클론
!git clone https://github.com/jud1thDev/Personalized-UI-Recommendation-System.git
%cd Personalized-UI-Recommendation-System

# 현재 Colab 환경에서 프로젝트 경로 설정
sys.path.append('/content/Personalized-UI-Recommendation-System')
print("프로젝트 경로 설정 완료")

## 2. 모듈 import 및 설정

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from datetime import datetime, timedelta
import random
import yaml
import os

warnings.filterwarnings('ignore')

# 프로젝트 모듈 import
from ui_rec.src.data.generate_mock import generate_users, generate_functions, simulate_events
from ui_rec.src.features.build_features import build_user_level, build_user_function, make_targets
from ui_rec.src.models.common import train_binary, train_multiclass, train_regression, split_xy
from ui_rec.src.models.ui_grouping import create_ui_component_groups
from ui_rec.src.utils.icons import get_icon_suggestion
from ui_rec.src.utils.io import ensure_dir, write_csv_with_timestamp, latest_file, save_model
from ui_rec.src.utils.time import access_time_cluster
from ui_rec.src.utils.features import entropy

print("모듈 import 완료")

## 3. 설정 파일 생성

In [ ]:
def create_config_files():
    # data.yaml 생성
    data_config = {
        "paths": {
            "raw_dir": "ui_rec/data/raw",
            "processed_dir": "ui_rec/data/processed", 
            "models_dir": "ui_rec/data/models",
            "logs_dir": "ui_rec/data/logs",
            "outputs_dir": "ui_rec/data/outputs"
        },
        "files": {
            "events_pattern": "events_*.csv",
            "features_pattern": "features_*.csv",
            "model_pattern": "*.joblib",
            "log_pattern": "*.log",
            "output_pattern": "ui_home_outputs.json"
        }
    }
    
    # model.yaml 생성
    model_config = {
        "lgbm": {
            "exposure": {
                "objective": "binary",
                "metric": "auc",
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "ui_type": {
                "objective": "multiclass",
                "metric": "multi_logloss",
                "num_class": 5,
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "service_cluster": {
                "objective": "multiclass",
                "metric": "multi_logloss",
                "num_class": 7,
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            },
            "rank": {
                "objective": "regression",
                "metric": "rmse",
                "boosting_type": "gbdt",
                "num_leaves": 31,
                "learning_rate": 0.05,
                "feature_fraction": 0.9,
                "bagging_fraction": 0.8,
                "bagging_freq": 5,
                "verbose": -1,
                "num_boost_round": 1000,
                "early_stopping_rounds": 50
            }
        },
        "ui": {
            "allowed_types": ["card", "list_item", "banner", "icon", "grid_item"],
            "default_type": "card"
        },
        "service_clusters": {
            "mapping": {
                "account": 0, "finance": 1, "lifestyle": 2, 
                "health": 3, "shopping": 4, "travel": 5, "security": 6
            },
            "labels": {
                "0": "account", "1": "finance", "2": "lifestyle",
                "3": "health", "4": "shopping", "5": "travel", "6": "security"
            }
        }
    }
    
    # 디렉토리 생성
    for path in data_config["paths"].values():
        os.makedirs(path, exist_ok=True)
    
    # 설정 파일 저장
    with open("ui_rec/config/data.yaml", "w", encoding="utf-8") as f:
        yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)
    
    with open("ui_rec/config/model.yaml", "w", encoding="utf-8") as f:
        yaml.dump(model_config, f, default_flow_style=False, allow_unicode=True)
    
    return data_config, model_config

print("설정 파일 생성 중...")
data_config, model_config = create_config_files()
print("설정 파일 생성 완료")

## 4. mock 데이터 생성

In [ ]:
# Mock 데이터 생성
print("모의 데이터 생성 중...")
users = generate_users(n_users=20)
funcs = generate_functions()
events = simulate_events(users, funcs, days=7)

# 데이터 저장
raw_file = write_csv_with_timestamp(events, "user_events", "ui_rec/data/raw")
print(f"모의 데이터 생성 완료: {len(users)}명 사용자, {len(events)}개 이벤트")
print(f"저장 위치: {raw_file}")

## 5. 피처 생성

In [ ]:
print("피처 생성 중...")

# 기존 build_features 모듈의 함수들을 사용하여 피처 생성
df = pd.read_csv(raw_file, parse_dates=["timestamp"])

# 사용자 레벨 피처 생성
user_df = build_user_level(df)

# 사용자-기능 단위 피처 생성
uf = build_user_function(df)

# 타깃 생성
targets = make_targets(uf, df)

# 피처 결합
meta = df.groupby("user_id")[["age_group", "is_senior", "device_type"]].agg(lambda x: x.iloc[0]).reset_index()
feat = uf.merge(user_df, on="user_id", how="left")
feat = feat.merge(meta, on="user_id", how="left")

# 타깃 병합
feat = feat.merge(targets.drop(columns=["entry_count","click_rate","visit_duration","return_count","last_access_days","rank_score"]),
                  on=["user_id","function_id"], how="left")

# 처리된 데이터 저장
processed_file = write_csv_with_timestamp(feat, "features", "ui_rec/data/processed")
print(f"피처 생성 완료: {len(feat)}개 샘플")
print(f"저장 위치: {processed_file}")

## 6. 모델 학습

In [ ]:
print("모델 학습 중...")

# 데이터 로드 및 전처리
df = pd.read_csv(processed_file)

# 학습/검증 데이터 분할
X, y_exposure = split_xy(df, "exposure_label")
_, y_ui_type = split_xy(df, "ui_type_label")
_, y_service_cluster = split_xy(df, "service_cluster_label")
_, y_rank = split_xy(df, "rank_label")

# Exposure 모델 학습
print("- Exposure 모델 학습 중...")
exposure_model = train_binary(df, "exposure_label", "exposure")
print(f"Exposure 모델 저장 완료")

# UI Type 모델 학습
print("- UI Type 모델 학습 중...")
ui_type_model = train_multiclass(df, "ui_type_label", "ui_type", "ui_type")
print(f"UI Type 모델 저장 완료")

# Service Cluster 모델 학습
print("- Service Cluster 모델 학습 중...")
service_cluster_model = train_multiclass(df, "service_cluster_label", "service_cluster", "service_cluster")
print(f"Service Cluster 모델 저장 완료")

# Rank 모델 학습
print("- Rank 모델 학습 중...")
rank_model = train_regression(df, "rank_label", "rank")
print(f"Rank 모델 저장 완료")

print("모델 학습 완료!")

## 7. 추론 및 결과 생성

In [ ]:
print("추론 및 결과 생성 중...")

# 모델 로드
exposure_model = joblib.load("ui_rec/data/models/exposure.joblib")
ui_type_model = joblib.load("ui_rec/data/models/ui_type.joblib")
service_cluster_model = joblib.load("ui_rec/data/models/service_cluster.joblib")
rank_model = joblib.load("ui_rec/data/models/rank.joblib")

# 추론용 데이터 준비
drop_cols = ["exposure_label", "ui_type_label", "service_cluster_label", "rank_label", "user_id", "function_id"]
X_pred = df.drop(columns=drop_cols)

# 예측 수행
exposure_pred = exposure_model.predict(X_pred)
ui_type_pred = ui_type_model.predict(X_pred)
service_cluster_pred = service_cluster_model.predict(X_pred)
rank_pred = rank_model.predict(X_pred)

# 결과 데이터프레임 생성
results_df = pd.DataFrame({
    "user_id": df["user_id"],
    "function_id": df["function_id"],
    "exposure": exposure_pred,
    "ui_type": ui_type_pred,
    "service_cluster": service_cluster_pred,
    "rank": rank_pred
})

# UI 타입 매핑
ui_type_mapping = {0: "card", 1: "list_item", 2: "banner", 3: "grid_item", 4: "icon"}
results_df["component_type"] = results_df["ui_type"].map(ui_type_mapping)

# 서비스 클러스터 매핑
results_df["service_cluster"] = results_df["service_cluster"].astype(str).map(model_config["service_clusters"]["labels"])

print("추론 완료!")

## 8. UI 그룹화 및 최종 결과 생성

In [ ]:
print("UI 그룹화 및 최종 결과 생성 중...")

# 노출될 기능만 필터링
exposed_functions = results_df[results_df["exposure"] == 1].copy()

# 아이콘 추천 추가 (기존 모듈 사용)
exposed_functions["icon_suggestion"] = exposed_functions.apply(
    lambda row: get_icon_suggestion(row["service_cluster"]), axis=1
)

# 사용자별로 그룹화하여 최종 결과 생성
final_results = []

for user_id in exposed_functions["user_id"].unique():
    user_functions = exposed_functions[exposed_functions["user_id"] == user_id].copy()
    
    # 순위별로 정렬
    user_functions = user_functions.sort_values("rank")
    
    # UI 컴포넌트 그룹 생성 (기존 모듈 사용)
    groups = create_ui_component_groups(user_functions.to_dict('records'), user_id)
    
    # 최종 결과 구조
    user_result = {
        "user_id": user_id,
        "home": {
            "layout_density": "medium",
            "groups": groups
        }
    }
    
    final_results.append(user_result)

print(f"UI 그룹화 완료: {len(final_results)}명 사용자")

## 9. 결과 저장 및 다운로드

In [ ]:
import json

# JSON 파일로 저장
output_file = "ui_rec/data/outputs/ui_home_outputs.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print(f"결과 저장 완료: {output_file}")

# 코랩에서 파일 다운로드
files.download(output_file)

print("\n" + "="*60)
print("Personalized UI Recommendation System 실행 완료!")
print(f"총 사용자: {len(final_results)}명")
print(f"결과 파일: {output_file}")
print("="*60)